In [1]:
import pandas as pd

from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [36]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'

sheet_name = 'incomati'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=incomati'

In [37]:
output_file_path = '../data/labels/gt-bands.xlsx'

In [38]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,SandSnapID,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,Incomati-1,-25.564683,32.209439,Sand,2023-05-08,fluvial,Author,Mozambique,Active mine site,1407.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Incomati-2,-25.564497,32.209442,Sand,2023-05-09,fluvial,Author,Mozambique,Active mine site,1419.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Incomati-3,-25.565606,32.210558,Sand,2023-05-10,fluvial,Author,Mozambique,NaN,1420.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Incomati-4,-25.572600,32.250383,Sand,2023-05-11,fluvial,Author,Mozambique,NaN,1421.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Incomati-5,-25.571472,32.251222,Sand,2023-05-12,fluvial,Author,Mozambique,Surface sample with a lot of gravel just on th...,1422.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Incomati-6,-25.571058,32.251406,Sand,2023-05-13,fluvial,Author,Mozambique,NaN,1423.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Incomati-7,-25.325008,32.254306,Sand,2023-05-14,fluvial,Author,Mozambique,Use the sandbank on the other side of the rive...,1425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Incomati-8,-25.513669,32.267183,Sand,2023-05-15,fluvial,Author,Mozambique,Active mine site,1426.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Incomati-9,-25.513669,32.267183,Sand,2023-05-16,fluvial,Author,Mozambique,Active mine site,1427.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Incomati-10,-25.441790,32.235749,Sand,2023-05-17,fluvial,Author,Mozambique,"Active mine site, not visited, sand trucks lea...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
#drop any columns that have 'Unnamed:' in the column name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,SandSnapID,SandSnap Photo,D16,D50,D90,Sieve Results,S-D16,S-D50,S-D90,SSE
0,Incomati-1,-25.564683,32.209439,Sand,2023-05-08,fluvial,Author,Mozambique,Active mine site,1407.0,PXL_20230508_065234336.jpg,0.507,0.707,1.150,True,0.3253,0.558716,1.055374,0.252898
1,Incomati-2,-25.564497,32.209442,Sand,2023-05-09,fluvial,Author,Mozambique,Active mine site,1419.0,PXL_20230508_065446465.jpg,0.490,0.684,1.113,True,0.3277,0.589959,1.178165,0.198574
2,Incomati-3,-25.565606,32.210558,Sand,2023-05-10,fluvial,Author,Mozambique,NaN,1420.0,PXL_20230508_070432316.jpg,0.651,0.908,1.478,False,NaN,NaN,NaN,NaN
3,Incomati-4,-25.572600,32.250383,Sand,2023-05-11,fluvial,Author,Mozambique,NaN,1421.0,PXL_20230508_073409598.jpg,0.541,0.756,1.231,True,0.3590,0.587000,1.045600,0.309933
4,Incomati-5,-25.571472,32.251222,Sand,2023-05-12,fluvial,Author,Mozambique,Surface sample with a lot of gravel just on th...,1422.0,PXL_20230508_074057140.jpg,0.560,0.782,1.273,True,NaN,NaN,NaN,1.595510
5,Incomati-6,-25.571058,32.251406,Sand,2023-05-13,fluvial,Author,Mozambique,NaN,1423.0,PXL_20230508_074347219.jpg,0.678,0.946,1.540,True,NaN,NaN,NaN,1.930337
6,Incomati-7,-25.325008,32.254306,Sand,2023-05-14,fluvial,Author,Mozambique,Use the sandbank on the other side of the rive...,1425.0,PXL_20230508_090836241.jpg,0.571,0.797,1.297,False,NaN,NaN,NaN,1.625872
7,Incomati-8,-25.513669,32.267183,Sand,2023-05-15,fluvial,Author,Mozambique,Active mine site,1426.0,PXL_20230508_081958555.jpg,0.664,0.929,1.513,False,NaN,NaN,NaN,1.895549
8,Incomati-9,-25.513669,32.267183,Sand,2023-05-16,fluvial,Author,Mozambique,Active mine site,1427.0,PXL_20230508_082125181.jpg,0.509,0.710,1.157,False,NaN,NaN,NaN,1.449769
9,Incomati-10,-25.441790,32.235749,Sand,2023-05-17,fluvial,Author,Mozambique,"Active mine site, not visited, sand trucks lea...",NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,0.000000


In [40]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [9]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [41]:
output = df.copy()

In [42]:
INDEX = 0

In [77]:
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [65]:
#Uncomment to change the date of the image
# output['Date'].iloc[INDEX] = '2022-10-14'
# output['ID'].iloc[INDEX] = 'ns-49'

In [78]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='median', sampling_buffer_m=sampling_buffer_m, 
                                 max_search_window_months=3, median_samples=5,roi_buffer_m=5000, obia=True)

Index:  11  ID:  Incomati-12 Class:  Sand  Site:  fluvial
Search window from 2023-02-19 to 2023-08-19


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function.
It will also overwrite just the sheet if the same exists, but will preserve the other sheets in the book

In [79]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
# fm.write_to_excel(output, output_file_path, sheet_name)
with pd.ExcelWriter(output_file_path,engine="openpyxl",mode="a",if_sheet_exists="replace") as writer:
    output.to_excel(writer,sheet_name=sheet_name,index=False)    
display(output.iloc[INDEX])
INDEX += 1

New marker accepted
Kept Observation


ID                                 Incomati-12
Latitude                             -25.28553
Longitude                            32.505906
Class                                     Sand
Date                                2023-05-19
Site                                   fluvial
Source                                  Author
Country                             Mozambique
Notes                Possibly active mine site
SandSnapID                              1430.0
SandSnap Photo      PXL_20230508_104747436.jpg
D16                                      0.662
D50                                      0.925
D90                                      1.506
Sieve Results                             True
S-D16                                      NaN
S-D50                                      NaN
S-D90                                      NaN
SSE                                   1.887301
B2_mean                            1313.319458
B3_mean                            1865.055542
B4_mean      

## Final Output

In [35]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,SandSnapID,...,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked,class_code
0,Incomati-1,-25.563461,32.207274,Sand,2023-05-08,fluvial,Author,Mozambique,Active mine site,1407.0,...,3766.500000,3439.946777,-13.139558,-22.845945,0.128520,0.178852,-0.256659,True,True,1
1,Incomati-2,-25.564497,32.209442,Sand,2023-05-09,fluvial,Author,Mozambique,Active mine site,1419.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
2,Incomati-3,-25.565606,32.210558,Sand,2023-05-10,fluvial,Author,Mozambique,NaN,1420.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
3,Incomati-4,-25.572375,32.250737,Sand,2023-05-11,fluvial,Author,Mozambique,NaN,1421.0,...,4786.509277,4542.254883,-13.276730,-23.325525,0.174733,0.220260,-0.250739,True,True,1
4,Incomati-5,-25.571325,32.251032,Sand,2023-05-12,fluvial,Author,Mozambique,Surface sample with a lot of gravel just on th...,1422.0,...,4786.509277,4542.254883,-12.517284,-23.633896,0.174733,0.220260,-0.250739,True,True,1
5,Incomati-6,-25.571058,32.251406,Sand,2023-05-13,fluvial,Author,Mozambique,NaN,1423.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
6,Incomati-7,-25.325008,32.254306,Sand,2023-05-14,fluvial,Author,Mozambique,Use the sandbank on the other side of the rive...,1425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
7,Incomati-8,-25.513669,32.267183,Sand,2023-05-15,fluvial,Author,Mozambique,Active mine site,1426.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
8,Incomati-9,-25.513669,32.267183,Sand,2023-05-16,fluvial,Author,Mozambique,Active mine site,1427.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
9,Incomati-10,-25.441790,32.235749,Sand,2023-05-17,fluvial,Author,Mozambique,"Active mine site, not visited, sand trucks lea...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,99
